# 14. 프로젝트 - Movielens 영화 추천 실습

🎬영화 추천 시스템

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. ```MovieLens 1M Dataset``` 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 **시청횟수**로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

data set
- movies.dat
- ratings.dat
- users.dat
- README

## 1) 데이터 준비와 전처리

Movielens 데이터는 ```rating.dat``` 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import pandas as pd
import os

In [2]:
readme=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/README'
f = open(readme, 'r')
lines = f.readlines()
for line in lines:
    print(line)
f.close()

SUMMARY




These files contain 1,000,209 anonymous ratings of approximately 3,900 movies 

made by 6,040 MovieLens users who joined MovieLens in 2000.



USAGE LICENSE




Neither the University of Minnesota nor any of the researchers

involved can guarantee the correctness of the data, its suitability

for any particular purpose, or the validity of results based on the

use of the data set.  The data set may be used for any research

purposes under the following conditions:



     * The user may not state or imply any endorsement from the

       University of Minnesota or the GroupLens Research Group.



     * The user must acknowledge the use of the data set in

       publications resulting from the use of the data set

       (see below for citation information).



     * The user may not redistribute the data without separate

       permission.



     * The user may not use this information for any commercial or

       revenue-bearing purposes without first obtaining permi

In [3]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [6]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
movies['title'] = movies['title'].str.lower() # 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


In [9]:
#성별, 나이, 직업, 우편번호
user_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/users.dat'
user_col = ['user_id', 'gender', 'age', 'occupation', 'zip-code']   
users = pd.read_csv(user_file_path, sep='::', names= user_col, engine='python', encoding = "ISO-8859-1") 
users.head(10)

,user_id,gender,age,occupation,zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


user특징 
- 성별/나이/직업/우편번호에 따라 분류해 놓았다.
- 우리에게 중요한 것은 ```user_id, movie_id, counts```이다.
- 사용자별 통계치는 중요한 데이터지만 나중에 다루기로 한다.


In [10]:
#사용하는 컬럼만 남겨줍니다.
#using_cols = ['user_id', 'artist', 'play']
#users = users[using_cols]
#users.head(10)

In [11]:
#첫번째 유저가 어떤 영화를 좋아하는지
condition = (ratings['user_id']== ratings.loc[0, 'user_id'])
ratings.loc[condition]

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


## 2) 데이터 분석해보기


In [12]:
#movie_id를 기준으로 데이터프레임 병합
merge_mov = pd.merge(ratings, movies, on='movie_id')
merge_mov.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),Drama
1,2,1193,5,978298413,one flew over the cuckoo's nest (1975),Drama
2,12,1193,4,978220179,one flew over the cuckoo's nest (1975),Drama
3,15,1193,4,978199279,one flew over the cuckoo's nest (1975),Drama
4,17,1193,5,978158471,one flew over the cuckoo's nest (1975),Drama


In [13]:
# ratings에 있는 유니크한 영화 개수
merge_mov['movie_id'].nunique()

3628

In [14]:
# ratings에 있는 유니크한 사용자 수
merge_mov['user_id'].nunique()

6039

In [15]:
# 인기 많은 영화
movie_count = merge_mov.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(20)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

- 별점 3점이상이면서 가장 많이 본 영화는 ```american beauty```이다.
- user수가 약 6000명이니 그 절반 이상이 좋게 평가한 영화이다.

In [16]:
# 한 영화당 몇명의 유저들이 봤는지 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [17]:
#유저 한명당 몇편의 영화를 보는지 통계
user_count = ratings.groupby('movie_id')['user_id'].count()
user_count.describe()

count    3628.000000
mean      230.561742
std       355.596393
min         1.000000
25%        23.000000
50%        87.000000
75%       285.000000
max      3211.000000
Name: user_id, dtype: float64

## 3) 내가 선호하는 영화를 5가지를 movie_data에 추가해보기

### 모델 검증을 위한 사용자 초기 정보 세팅

추천 시스템
- 처음 가입하는 사용자의 취향과 유사한 정보를 5개 이상 입력받는 과정을 거치게 하는 경우가 많다.<br>(본인의 취향과 가장 유사한 영화 추천)
- 우리가 만든 추천 시스템의 추후 검증 과정을 위해, '나는 A를 좋아한다.'라는 정보를 위 데이터셋에 5개 이상 추가해 주어야 합니다. 
- 내가 좋아하는 영화들을 기존의 데이터에 추가해 넣기(*2000년대 이전의 영화*)

In [18]:
cols = ['user_id', 'title', 'counts']
movie_data = merge_mov[cols]
movie_data.head()

,user_id,title,counts
0,1,one flew over the cuckoo's nest (1975),5
1,2,one flew over the cuckoo's nest (1975),5
2,12,one flew over the cuckoo's nest (1975),4
3,15,one flew over the cuckoo's nest (1975),4
4,17,one flew over the cuckoo's nest (1975),5


In [19]:
temp_titles = merge_mov['title'].str.lower()
merge_mov[['title']][temp_titles.str.contains('beauty')]
merge_mov[['title']][temp_titles.str.contains('before')]
merge_mov[['title']][temp_titles.str.contains('pinocchio')]

,title
369500,pinocchio (1940)
369501,pinocchio (1940)
369502,pinocchio (1940)
369503,pinocchio (1940)
369504,pinocchio (1940)
...,...
832592,"adventures of pinocchio, the (1996)"
832593,"adventures of pinocchio, the (1996)"
832594,"adventures of pinocchio, the (1996)"
832595,"adventures of pinocchio, the (1996)"


In [20]:
#좋아하는 영화 5개
my_favorite = ['beauty and the beast (1991)' , 'Back to the Future (1985)' ,'Toy Story (1995)' ,
               'before sunrise (1995)' ,'pinocchio (1940)']

my_list = pd.DataFrame({'user_id': [6041]*5, 'title': my_favorite, 'counts':[5,3,4,4,5]})

if not movie_data.isin({'user_id':[6041]})['user_id'].any():  # user_id에 '6041'이라는 데이터가 없다면
    data = movie_data.append(my_list)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,counts
836473,5851,one little indian (1973),5
836474,5854,slaughterhouse (1987),4
836475,5854,"promise, the (versprechen, das) (1994)",3
836476,5938,"five wives, three secretaries and me (1998)",4
836477,5948,identification of a woman (identificazione di ...,5
0,6041,beauty and the beast (1991),5
1,6041,Back to the Future (1985),3
2,6041,Toy Story (1995),4
3,6041,before sunrise (1995),4
4,6041,pinocchio (1940),5


# 4) CSR matrix를 직접 만들어 봅시다.

### 모델에 활용하기 위한 전처리

- indexing 작업
- 추천 시스템, 자연어 처리에서 자주 하는 작업들
- 데이터의 관리를 쉽게 하기 위해! 우리가 다루는 데이터에서는 user와 title 각각에 번호를 붙이면 된다.  

- ```pandas.DataFrame.unique()```은 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. indexing 작업을 위해 매우 유용합니다.

In [21]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [22]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[6041])    
print(title_to_idx['beauty and the beast (1991)'])

6039
10


In [23]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,10,5
1,6039,3628,3
2,6039,3629,4
3,6039,213,4


## 14-6. CSR(Compressed Sparse Row) Matrix

In [24]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3630 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.


Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해 봅시다.

- ```implicit``` 패키지는 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있다.
- 이 패키지의 ```als(AlternatingLeastSquares) 모델```은
- ```Matrix Factorization```에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식

In [25]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 ```__init__``` 파라미터를 살펴보겠습니다.

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1과 4를 늘릴수록 학습 데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [26]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=50, dtype=np.float32)

In [27]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3630x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [28]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

모델 학습이 끝났습니다! 이제 2가지를 아래 사항을 살펴보도록 하겠습니다.

1. 6041 벡터와 beauty and the beast 의 벡터를 어떻게 만들고 있는지
2. 두 벡터를 곱하면 어떤 값이 나오는지

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.


In [29]:
user6041, beast = user_to_idx[6041], title_to_idx['beauty and the beast (1991)']
user6041_vector, beast_vector = als_model.user_factors[user6041], als_model.item_factors[beast]

In [30]:
user6041_vector

array([-0.00715649,  0.07746248, -0.18662377, -0.35272038, -0.05903573,
        0.42178425, -0.2058486 , -0.03021776,  0.25789624, -0.23612465,
        0.31815323,  0.35091108, -0.28213713,  0.01925837, -0.16640483,
        0.13298747, -0.22518133, -0.12524968, -0.03743031,  0.03968112,
        0.14133136,  0.24462724,  0.0535132 , -0.13213256,  0.19247538,
        0.01135529,  0.4149704 , -0.2738701 ,  0.09273124, -0.25025395,
       -0.02485406,  0.19126105,  0.19611055, -0.37667918,  0.2614404 ,
       -0.04139135,  0.2590755 , -0.19731863,  0.07679452,  0.1648573 ,
       -0.34247944,  0.0868851 ,  0.09511589,  0.02170886,  0.23218207,
        0.44357353,  0.04250064,  0.00756689, -0.3956353 ,  0.17009933,
        0.01590787,  0.3855569 , -0.19024986, -0.28226322, -0.03391667,
       -0.3720141 ,  0.48691383,  0.07515792,  0.07209565, -0.47375387,
       -0.03001277, -0.07407966,  0.36684164,  0.43602687, -0.00821002,
       -0.28042293, -0.0365045 ,  0.3533379 , -0.07507563, -0.12

In [31]:
beast_vector

array([ 0.0080989 ,  0.01381998, -0.01756625,  0.01157518,  0.00777213,
        0.03716855, -0.02757098,  0.01966676,  0.02999595, -0.0050835 ,
        0.04527861,  0.04203375, -0.00080035, -0.00878834, -0.00754321,
        0.01741148, -0.02955998, -0.03048432, -0.00098648,  0.02165568,
        0.01306525,  0.01661245, -0.01258347, -0.00250935,  0.04091736,
       -0.01695563,  0.02721353, -0.01503646,  0.00708202, -0.01214874,
        0.00204223, -0.00123749,  0.01983229, -0.04652639,  0.02696845,
       -0.00939153,  0.04159104, -0.03194957,  0.01343736,  0.01333431,
       -0.01479965,  0.04089746, -0.01550459,  0.02014306,  0.01794961,
        0.03126036,  0.01365234,  0.01813899, -0.03794048,  0.0007475 ,
       -0.00493197,  0.04580604, -0.00794308, -0.04043281,  0.00673219,
       -0.02960379,  0.05882555,  0.02817988, -0.00879283, -0.04482541,
       -0.00674242,  0.03066926,  0.04436649,  0.04043885,  0.00460699,
        0.00507353, -0.00812139,  0.03082666,  0.00759026, -0.00

In [32]:
#user6041과 beauty and the beast을 내적하는 코드
np.dot(user6041_vector, beast_vector)

0.40847436

user6041의 리스트안에 미녀와 야수를 넣었지만 0.4의 수치가 나왔다.<br> 
이 수치만으로는 모델이 잘 학습한 것인지 알기는 어렵다.<br>
다음 다른 영화에 대한 선호도를 어떻게 예측할지 볼것이다.

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [33]:
favorite_movie = 'truman show, the (1998)'
title_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(title_id, N=15)
similar_movie

[(385, 1.0000001),
 (39, 0.6565895),
 (641, 0.6045274),
 (465, 0.5969098),
 (248, 0.596067),
 (386, 0.5660268),
 (406, 0.5530257),
 (533, 0.53239936),
 (843, 0.49337706),
 (384, 0.4821683),
 (437, 0.47790486),
 (429, 0.47303885),
 (844, 0.47255087),
 (18, 0.463272),
 (556, 0.45164615)]

In [34]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['truman show, the (1998)',
 'apollo 13 (1995)',
 "mr. holland's opus (1995)",
 'quiz show (1994)',
 'good will hunting (1997)',
 'philadelphia (1993)',
 'people vs. larry flynt, the (1996)',
 'fried green tomatoes (1991)',
 'primary colors (1998)',
 'jerry maguire (1996)',
 "what's eating gilbert grape (1993)",
 'scent of a woman (1992)',
 'rainmaker, the (1997)',
 'awakenings (1990)',
 'river runs through it, a (1992)']

In [35]:
def get_similar_movie(movie_name: str):
    title_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(title_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [36]:
get_similar_movie('home alone (1990)')

['home alone (1990)',
 'sister act (1992)',
 'mrs. doubtfire (1993)',
 'mighty ducks, the (1992)',
 'home alone 2: lost in new york (1992)',
 'father of the bride part ii (1995)',
 'liar liar (1997)',
 '101 dalmatians (1996)',
 'my blue heaven (1990)',
 'first wives club, the (1996)']

In [37]:
get_similar_movie('good will hunting (1997)')

['good will hunting (1997)',
 'shawshank redemption, the (1994)',
 'apollo 13 (1995)',
 'truman show, the (1998)',
 'dead man walking (1995)',
 'pulp fiction (1994)',
 'awakenings (1990)',
 'jerry maguire (1996)',
 "what's eating gilbert grape (1993)",
 'searching for bobby fischer (1993)']

나홀로 집에 home alone (1990)

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

- ```AlternatingLeastSquares``` 클래스에 구현되어 있는 ```recommend``` 메서드를 통하여 유저가 좋아할만한 영화를 추천한다. 
- ```filter_already_liked_items``` 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [38]:
user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(330, 0.34028113),
 (33, 0.30273837),
 (8, 0.29311073),
 (37, 0.28811067),
 (17, 0.287292),
 (191, 0.2634983),
 (46, 0.2613876),
 (572, 0.24521695),
 (547, 0.22418325),
 (551, 0.21633841),
 (548, 0.21235451),
 (536, 0.2082034),
 (950, 0.20104265),
 (619, 0.20032705),
 (34, 0.19611911),
 (30, 0.18888333),
 (16, 0.18053202),
 (520, 0.17024726),
 (4, 0.15865356),
 (45, 0.15162943)]

In [39]:
[idx_to_title[i[0]] for i in movie_recommended]

['lion king, the (1994)',
 'aladdin (1992)',
 'snow white and the seven dwarfs (1937)',
 'cinderella (1950)',
 'bambi (1942)',
 'little mermaid, the (1989)',
 'dumbo (1941)',
 'sleeping beauty (1959)',
 'lady and the tramp (1955)',
 'peter pan (1953)',
 'fantasia (1940)',
 'jungle book, the (1967)',
 '101 dalmatians (1961)',
 'alice in wonderland (1951)',
 'mulan (1998)',
 'antz (1998)',
 'tarzan (1999)',
 "charlotte's web (1973)",
 "bug's life, a (1998)",
 'mary poppins (1964)']

favorite 리스트 안에 3편의 애니메이션을 넣어서 그런지 전부 애니메이션인 추천목록이 나온다<br>
추천 목록을 보면 영화간의 유사성은 잘 파악하는 것으로 보인다.<br>
좀더 다양한 장르로 추천해줬으면 하는 아쉬움이 있다.

- ```AlternatingLeastSquares``` 클래스에 구현된 ```explain``` 메서드를 사용하면 
- 5가지 고른 데이터 중 이 추천에 기여한 정도를 확인할 수 있습니다.

In [40]:
bambi = title_to_idx['bambi (1942)']
explain = als_model.explain(user, csr_data, itemid=bambi)

In [41]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('pinocchio (1940)', 0.20043388997101874),
 ('beauty and the beast (1991)', 0.07158834595831849),
 ('before sunrise (1995)', 0.012103554364486147),
 ('Toy Story (1995)', 0.0007822762827913866),
 ('Back to the Future (1985)', 0.0004398140981171245)]

### * (추가)

성별과 나이에 따라서 어떻게 다르게 나올까

### 1. 데이터 준비

In [42]:
# 성별, 나이, 직업, 우편번호
user_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/users.dat'
user_col = ['user_id', 'gender', 'age', 'occupation', 'zip-code']   
users = pd.read_csv(user_file_path, sep='::', names= user_col, engine='python', encoding = "ISO-8859-1")
users.head(10)

,user_id,gender,age,occupation,zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [43]:
users = users[users['gender']=='M']
filtered_data_size = len(users)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 4331
Ratio of Remaining Data is 0.43%


In [44]:
for c in users.columns:
    print('{} : {}'.format(c, len(users.loc[pd.isnull(users[c]), c].values)))

user_id : 0
gender : 0
age : 0
occupation : 0
zip-code : 0


성별 f, m모두 5000이하의 데이터크기를 가지고 있다.<br>
그렇다면 f, m을 제외한 나머지는 null의 값을 가지는지 궁금해서 알아보았지만
null값을 가진 것은 하나도 없었다.<br>
아마도 다른 방식으로 표기가 된 것 같다.

> X

### * (추가_end)


# 루브릭

아래의 기준을 바탕으로 프로젝트를 평가합니다.

|평가문항||상세기준|
|:----||:-----|
|1. CSR matrix가 정상적으로 만들어졌다.||사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|
|2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.||사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
|3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.||MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|

# 회고

추천시스템
- 이번껀 상당히 재밌었다.
- 저번에는 bert가 어떤 모델구조를 가진지 이해를 잘 못하고 했던 반면, 이번은 추천시스템의 전반적인 지식을 알고 더 차근차근히 잘볼 수 있었던것 같다.(A**의 역계단식 학습법..ㅂㄷㅂㄷ)
- 재밌었던 점은 **명시적/암묵적 평가**가 사용자의 선호도를 판단하는 지표인데, 명시적평가는-좋아요나 별점처럼 사용자의 선호도를 직접적으로 보여주는 것이라면 암묵적평가는-사용자가 영상을 몇번 보았나, 얼마나 봤나와 같은 간접적인 선호도를 나타낸다. 우리가 이번에 해본 프로젝트는 별점 개수를 3개 이상인 데이터만 추려내어 플레이횟수로 인식해서 진행했다.
- 이 점에서 우리가 가진 데이터셋을 단순하게 다양하게 변형이 가능하기도 하고, 이런 암묵적 평가를 어떻게 인식하느냐에 따라 모델의 결과가 다르게 나올 것이란걸 알게되었다.
- 그리고 노드 중간중간에 질문을 던지는 ```Q. (ex. 어떤 아티스트의 곡을 한번만 들은 유저에겐 그 아티스트와 관련된 사람을 추천해야할까?)```와 같은 답이 존재하지 않는 심심치않은 질문들로 흥미를 돋구게 한 내용이었던 것 같다.

https://danthetech.netlify.app/DataScience/evaluation-metrics-for-recommendation-system